## testing for feature engineering functions

In [2]:
!pip install /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
from msspackages import setup_runner
setup_runner(setup_type = 'notebook' , project = 'understanding-eks-data')

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/git/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
  Using cached pyspark-3.3.1-py2.py3-none-any.whl
  Using cached configparser-5.3.0-py3-none-any.whl (19 kB)
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
b"Get:1 http://deb.debian.org/debian buster InRelease [122 kB]\nGet:2 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nGet:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]\nGet:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [422 kB]\nGet:5 http://deb.debian.org/debian buster/main amd64 Packages [7909 kB]\nGet:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]\nFetched 8553 kB in 1s (6543 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n57 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding depende

In [3]:
from pyspark.ml.feature import VectorAssembler

In [15]:
import numpy as np
import pandas as pd
import random
#from ..utilities import feature_processor, null_report
from msspackages import Pyspark_data_ingestion, get_features

from sklearn.preprocessing import StandardScaler

from pyspark.sql.functions import col, count
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as f
from pyspark.sql import Window

resolved by os.chdir()

## Preprocess

In [16]:

def node_hmm_fe_v2(feature_group_name, feature_group_version, input_year, input_month, input_day, input_hour, input_setup = "default"):

    node_data = Pyspark_data_ingestion(
        year = input_year, 
        month = input_month, 
        day = input_day, 
        hour = input_hour, 
        setup = input_setup, 
        filter_column_value ='Node')
    err, node_df = node_data.read()
    node_df = node_df.select("InstanceId",'Timestamp','node_cpu_utilization','node_memory_utilization')

 
    if err == 'PASS':
        
        #get features
        features_df = get_features(feature_group_name,feature_group_version)
        features = features_df["feature_name"].to_list()
        processed_features = feature_processor.cleanup(features)
        
        model_parameters = features_df["model_parameters"].iloc[0]
  
        #drop na values in node cpu and memory utilization
        node_df = node_df.select("InstanceId","Timestamp", *processed_features)
        node_df = node_df.na.drop(subset=processed_features)
        
        #remove nodes which has a time gap over 2 minutes (epochtime = 2*60*1000=120000)
        w = Window.partitionBy('InstanceId').orderBy('Timestamp')
        node_df = node_df.withColumn('lead', f.lag('Timestamp', 1).over(w)) \
              .withColumn(
                'Timediff', 
                f.when(f.col('lead').isNotNull(), 
                f.col('Timestamp') - f.col('lead'))
                .otherwise(f.lit(None)))
               
        
        temp_df = node_df\
            .groupby("InstanceId")\
            .max("Timediff")\
            .select('InstanceId',f.col('max(TimeDiff)').alias('maxDiff'))\
            .filter("maxDiff<=120000")
                                                             
        node_df = node_df.filter(col("InstanceId").isin(temp_df['InstanceId']))
        node_df = node_df.sort("InstanceId","Timestamp")
        node_df = node_df.select('InstanceId','Timestamp','node_cpu_utilization','node_memory_utilization')
        
        #Drop rows with nans 
        node_df = node_df.na.drop("all")
           
        
        return node_df
    
    else:
        empty_df = pd.DataFrame()
        return empty_df

### Test code trunks in the function defined above

In [17]:
node_data = Pyspark_data_ingestion(
    year =2022,
    month = 7, 
    day = 10, 
    hour = 10, 
    setup = '128gb', 
    filter_column_value ='Node')
err, node_df = node_data.read()
node_df = node_df.select("InstanceId",'Timestamp','node_cpu_utilization','node_memory_utilization')

In [18]:
processed_features = ['node_cpu_utilization','node_memory_utilization']
node_df = node_df.na.drop(subset=processed_features)

In [19]:
#remove nodes which has a time gap over 2 minutes (epochtime = 2*60*1000=120000)
w = Window.partitionBy('InstanceId').orderBy('Timestamp')
node_df = node_df.withColumn('lead', f.lag('Timestamp', 1).over(w)) \
      .withColumn(
        'Timediff', 
        f.when(f.col('lead').isNotNull(), 
        f.col('Timestamp') - f.col('lead'))
        .otherwise(f.lit(None)))
 

In [20]:

temp_df = node_df\
    .groupby("InstanceId")\
    .max("Timediff")\
    .select('InstanceId',f.col('max(TimeDiff)').alias('maxDiff'))\
    .filter("maxDiff<=120000")

In [21]:
node_df = node_df.filter(col("InstanceId").isin(temp_df['InstanceId']))
node_df = node_df.sort("InstanceId","Timestamp")
node_df = node_df.select('InstanceId','Timestamp','node_cpu_utilization','node_memory_utilization')
        
#Drop rows with nans 
node_df = node_df.na.drop("all")

## Train_Test_Split

In [22]:
def node_hmm_train_test_split(input_df,split = 0.5):
    
    temp_df = input_df.select('InstanceId')
    node_train_id, node_test_id = temp_df.randomSplit(weights=[split,1-split], seed=200)  
    node_train = input_df.filter(col("InstanceId").isin(node_train_id['InstanceId']))
    node_test = input_df.filter(col("InstanceId").isin(node_test_id['InstanceId']))
    
    return node_train, node_test
    

    

In [33]:
node_train, node_test = node_hmm_train_test_split(node_df,split = 0.5)

#### test function

In [32]:
node_df.columns

['InstanceId', 'Timestamp', 'node_cpu_utilization', 'node_memory_utilization']

In [ ]:
temp_df = node_df.select('InstanceId')

In [25]:
node_train, node_test = temp_df.randomSplit(weights=[0.5,0.5], seed=200)

## Feature Engineering

In [37]:
def node_hmm_feature_engineer(input_df):
    
    #sort data
    input_df = input_df.sort('InstanceId','Timestamp')
    
    #get features
    features_df = get_features(feature_group_name,feature_group_version)
    features = features_df["feature_name"].to_list()
    
    #standardize feature data from the node
    features = ['node_cpu_utilization','node_memory_utilization']
    w = Window.partitionBy('InstanceId')
    for c in features:
        input_df = (input_df.withColumn('mean', f.min(c).over(w))
            .withColumn('std', f.max(c).over(w))
            .withColumn(c, ((f.col(c) - f.col('mean')) / (f.col('std'))))
            .drop('mean')
            .drop('std'))
        
    #standard scale the data
    vecAssembler = VectorAssembler(inputCols=["node_cpu_utilization", "node_memory_utilization"], outputCol="features")
    node_train = vecAssembler.transform(node_train)
    node_train = node_train.select('InstanceId','features')
        
    #transfer data to a nested list (#timestamps * #features for each node)
    instance_list = [x.InstanceId for x in node_train.select('InstanceId').distinct().collect()]
    features_list = []
    for instance in instance_list:
        sub = node_train.filter(node_train.InstanceId == instance)
        sub_features = np.array(sub.select("features").collect())
        features_list.append(sub_features)
                                
    return features_list
    
               

#### test code

In [39]:
features = ['node_cpu_utilization','node_memory_utilization']
w = Window.partitionBy('InstanceId')
for c in features:
    node_train = (node_train.withColumn('mean', f.mean(c).over(w))
        .withColumn('std', f.stddev(c).over(w))
        .withColumn(c, ((f.col(c) - f.col('mean')) / (f.col('std'))))
        .drop('mean')
        .drop('std'))
               

In [ ]:
node_train.columns


['InstanceId', 'Timestamp', 'node_cpu_utilization', 'node_memory_utilization']

In [69]:
instance_list = [x.InstanceId for x in node_train.select('InstanceId').distinct().collect()]

In [97]:
features_list = []

In [78]:
for instance in instance_list:
    sub = node_train.filter(node_train.InstanceId == instance)
    x = sub.select(*features)
    features_list.append(x)
    
                                

In [80]:
features_list[0].show()

+--------------------+-----------------------+
|node_cpu_utilization|node_memory_utilization|
+--------------------+-----------------------+
|  1.0399082431241218|     1.3805902749605883|
| -0.8748158495639415|     2.0164509111755637|
|  0.5902901236603855|    -1.1441504864811822|
|  1.5381148653378476|    -0.5303919579420007|
|   0.508159661093178|    -0.7531131968302485|
|  0.6417767790803964|    -0.3433741237611836|
|  1.6581373667192867|   0.001758788409084...|
|  0.5090572528104451|   0.061264462921140704|
|  1.0469963939423144|    -0.7905167636665105|
|  0.8256987187162563|    0.41319802360673474|
| 0.37373691219606753|     0.2227798651680084|
|   1.917564205192047|     1.6849192960366854|
|   0.471220442025022|   -0.14955564106479446|
|   1.021690767924856|     -0.305970556925168|
| -0.4168038744542885|   0.020460571826969497|
|-0.42091314387018436|      0.717527044682832|
|   0.578657381022802|     1.1017636858181938|
| -0.6081726869817168|    -0.7514130347014171|
| -0.38420012

In [ ]:
 # python list
np.array(x)

In [ ]:
instance_list = [x.InstanceId for x in node_train.select('InstanceId').distinct().collect()]
features_list = []
for instance in instance_list:
    sub = node_train.filter(node_train.InstanceId == instance)
    sub_features = np.array(sub.select(*features).collect())
    features_list.append(sub_features)
                                

In [85]:
nodes_pd = node_train.toPandas()

In [98]:
for i in instance_list:
    features_list.append(np.array(nodes_pd.loc[nodes_pd.InstanceId == i][features]))

In [99]:
nodes_pd.head()


,InstanceId,Timestamp,node_cpu_utilization,node_memory_utilization
0,i-00019e61765dd283f,1657447613053,0.098077,1.327352
1,i-00019e61765dd283f,1657447679159,0.921836,-0.129462
2,i-00019e61765dd283f,1657447731697,-0.975469,-0.005124
3,i-00019e61765dd283f,1657447802526,-0.076023,0.683142
4,i-00019e61765dd283f,1657447862267,-1.817445,0.485376


# model building

features_list

In [105]:
len(features_list[1])

58

In [109]:
!pip install hmmlearn

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached hmmlearn-0.2.8-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (217 kB)


In [110]:
from hmmlearn import hmm

In [145]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full", init_params = 'mc')
#model.startprob_ = [1.00000000e+00,1.40157062e-10,5.10958183e-25]
#model.transmat_ = [[0.22629484,0.59814545,0.17555972],[0.10805554,0.5319676,0.35997686],[0.47684518,0.00247531,0.52067951]]
model.fit(features_list[0])

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
            covars_weight=1, implementation='log', init_params='mc',
            means_prior=0, means_weight=0, min_covar=0.001, n_components=3,
            n_iter=10, params='stmc', random_state=None, startprob_prior=1.0,
            tol=0.01, transmat_prior=1.0, verbose=False)

In [146]:
print(model.startprob_,model.transmat_)

[9.99999999e-01 1.30831615e-26 1.06117568e-09] [[1.80060090e-01 8.16175961e-01 3.76394832e-03]
 [2.33371098e-04 7.29240095e-01 2.70526534e-01]
 [7.08621280e-01 2.62111440e-01 2.92672804e-02]]


In [155]:
start_prob = [9.99999999e-01, 1.30831615e-26, 1.06117568e-09]
transmate_matrix = [[1.80060090e-01, 8.16175961e-01, 3.76394832e-03],
 [2.33371098e-04, 7.29240095e-01, 2.70526534e-01],
 [7.08621280e-01, 2.62111440e-01, 2.92672804e-02]]
model = hmm.GaussianHMM(n_components=3, covariance_type="full", init_params = 'mc')
for i in range(1,len(features_list)):
    
    model.startprob_ = start_prob
    model.transmat_ = transmate_matrix
    model.fit(features_list[i])
    start_prob = model.startprob_
    transmat_matrix = model.transmat_ 
        
    print(start_prob,transmat_matrix)

[1.00000000e+00 1.18209964e-31 5.84956314e-18] [[3.30606862e-01 6.68229477e-01 1.16366028e-03]
 [1.26639449e-03 3.13667684e-01 6.85065921e-01]
 [7.52748817e-01 2.46527019e-01 7.24163360e-04]]
[1.00000000e+00 3.09252303e-51 1.87165021e-36] [[3.45358219e-01 6.54641762e-01 1.82922477e-08]
 [3.90926030e-04 8.07636267e-01 1.91972807e-01]
 [6.36672073e-01 2.11477312e-02 3.42180196e-01]]
[1.00000000e+000 4.30656518e-030 6.58254524e-164] [[8.09003948e-01 1.90995453e-01 5.98456701e-07]
 [4.43695661e-08 1.82290196e-01 8.17709760e-01]
 [1.25103694e-01 2.06309952e-01 6.68586354e-01]]
[1.00000000e+000 2.83084653e-066 2.34218092e-240] [[8.31770110e-02 1.37799393e-01 7.79023596e-01]
 [2.20834609e-07 9.99998512e-01 1.26715250e-06]
 [5.20017030e-01 1.37501915e-03 4.78607951e-01]]
[1.00000000e+000 6.14480215e-069 4.75888320e-273] [[5.50476774e-03 9.94474511e-01 2.07212648e-05]
 [2.49618312e-05 7.66024483e-01 2.33950555e-01]
 [9.38813154e-01 3.56189468e-02 2.55678988e-02]]
[1.00000000e+00 4.96621275e-81 

[1. 0. 0.] [[8.38215610e-01 1.56285273e-01 5.49911639e-03]
 [4.79380073e-06 7.13205406e-01 2.86789800e-01]
 [2.75602628e-01 2.61171973e-01 4.63225400e-01]]
[1. 0. 0.] [[8.91927388e-01 7.89293369e-02 2.91432754e-02]
 [4.34381227e-04 3.56911829e-02 9.63874436e-01]
 [2.31611982e-01 6.69839357e-01 9.85486611e-02]]
[1. 0. 0.] [[7.27879300e-01 2.32000609e-01 4.01200910e-02]
 [2.26422884e-04 5.42734946e-01 4.57038631e-01]
 [3.38516209e-01 6.30489611e-01 3.09941800e-02]]
[1. 0. 0.] [[4.83282216e-01 5.16047824e-01 6.69960537e-04]
 [4.21068680e-04 6.47569369e-01 3.52009562e-01]
 [7.45373435e-01 1.40418937e-02 2.40584672e-01]]
[1. 0. 0.] [[2.54848097e-01 7.82892714e-03 7.37322976e-01]
 [2.50753637e-19 1.00000000e+00 8.10295287e-12]
 [7.90540657e-01 6.28525804e-02 1.46606763e-01]]
[1. 0. 0.] [[3.58359785e-04 9.23786486e-01 7.58551541e-02]
 [7.56584725e-04 5.46907140e-01 4.52336276e-01]
 [9.16595498e-01 6.31102793e-02 2.02942224e-02]]
[1. 0. 0.] [[3.19915940e-01 6.58386293e-01 2.16977661e-02]
 [7.1

[1. 0. 0.] [[0.73505233 0.02559834 0.23934933]
 [0.00106488 0.         0.99893512]
 [0.27179776 0.00930274 0.71889951]]
[1. 0. 0.] [[5.16330899e-01 3.80520630e-01 1.03148471e-01]
 [1.56522619e-05 1.77626801e-10 9.99984348e-01]
 [8.43763651e-01 5.74767254e-06 1.56230601e-01]]
[1. 0. 0.] [[9.37563506e-01 6.24364943e-02 2.17162366e-39]
 [2.04545824e-13 9.72959281e-01 2.70407195e-02]
 [8.27782821e-10 2.50052873e-01 7.49947126e-01]]
[1. 0. 0.] [[3.38409791e-51 1.00000000e+00 0.00000000e+00]
 [4.85460094e-05 9.81768753e-01 1.81827008e-02]
 [1.00000000e+00 8.53672588e-12 0.00000000e+00]]
[1. 0. 0.] [[8.34293819e-01 6.19033397e-04 1.65087147e-01]
 [4.63815774e-05 8.83087610e-01 1.16866009e-01]
 [1.30726758e-01 3.36570078e-01 5.32703164e-01]]
[1. 0. 0.] [[8.86124357e-01 1.37422450e-09 1.13875641e-01]
 [4.99999983e-01 5.00000000e-01 1.73347261e-08]
 [7.72056232e-02 4.48189680e-02 8.77975409e-01]]
[1. 0. 0.] [[2.22997752e-02 9.77700224e-01 7.99610002e-10]
 [1.76531853e-03 9.25941438e-01 7.2293243

[1. 0. 0.] [[9.65572589e-01 3.60801493e-11 3.44274113e-02]
 [1.36179222e-30 9.33716498e-01 6.62835025e-02]
 [7.05541048e-20 7.88503594e-02 9.21149641e-01]]
[1. 0. 0.] [[4.63472345e-01 5.34229825e-01 2.29783034e-03]
 [6.05762157e-04 5.70515129e-01 4.28879109e-01]
 [7.76894998e-01 1.16241752e-01 1.06863250e-01]]
[1. 0. 0.] [[1.65649794e-01 8.20877343e-01 1.34728636e-02]
 [1.24850620e-05 8.32898277e-01 1.67089238e-01]
 [5.35315113e-01 4.44424157e-02 4.20242471e-01]]
[1. 0. 0.] [[0.07377034 0.86253267 0.06369699]
 [0.08318139 0.45614815 0.46067047]
 [0.69255701 0.02169043 0.28575256]]
[1. 0. 0.] [[4.86785237e-02 9.51315472e-01 6.00468948e-06]
 [2.12969062e-04 7.02952879e-01 2.96834152e-01]
 [1.03419965e-01 8.96569176e-01 1.08597835e-05]]
[1. 0. 0.] [[7.59012322e-01 2.40987678e-01 3.98243542e-13]
 [3.49874083e-01 5.35771693e-01 1.14354225e-01]
 [9.99999009e-01 9.90765987e-07 1.60636195e-44]]
[1. 0. 0.] [[8.96127927e-01 6.20053587e-02 4.18667138e-02]
 [6.73548064e-01 3.38260849e-29 3.2645193

[1. 0. 0.] [[4.98442923e-01 5.01552661e-01 4.41597022e-06]
 [1.26548748e-05 4.70933717e-01 5.29053628e-01]
 [6.97388143e-01 6.69679024e-02 2.35643955e-01]]
[1. 0. 0.] [[0.8214284  0.10714302 0.07142858]
 [0.12000015 0.83999985 0.04      ]
 [0.49999988 0.25000012 0.25      ]]
[1. 0. 0.] [[7.21474286e-01 6.76317331e-02 2.10893981e-01]
 [3.27084140e-14 9.50000000e-01 5.00000000e-02]
 [9.74207017e-01 3.54423858e-07 2.57926285e-02]]
[1. 0. 0.] [[1.28284723e-02 9.87164420e-01 7.10723304e-06]
 [1.99185188e-04 7.63349435e-01 2.36451380e-01]
 [9.30707685e-01 6.33835827e-02 5.90873249e-03]]
[1. 0. 0.] [[0.27085535 0.71381596 0.01532868]
 [0.00132889 0.62412492 0.3745462 ]
 [0.51944948 0.47319414 0.00735639]]
[1. 0. 0.] [[6.50783181e-01 3.49131057e-01 8.57619395e-05]
 [1.83261684e-01 7.87171493e-01 2.95668227e-02]
 [9.99601741e-01 3.98258704e-04 0.00000000e+00]]
[1. 0. 0.] [[8.32814839e-01 1.67185161e-01 8.30051795e-83]
 [1.08157442e-06 9.79598370e-01 2.04005481e-02]
 [6.05110150e-09 9.99999994e-

...

[1. 0. 0.] [[8.07364866e-01 1.92581193e-01 5.39416158e-05]
 [5.45805509e-02 6.86882973e-01 2.58536476e-01]
 [4.68444588e-01 5.01319690e-01 3.02357222e-02]]
[1. 0. 0.] [[1.12031079e-05 7.11998917e-01 2.87989880e-01]
 [4.11083026e-03 3.48640679e-01 6.47248491e-01]
 [4.70857076e-01 5.14195395e-01 1.49475285e-02]]
[1. 0. 0.] [[9.11448518e-02 6.98127281e-01 2.10727867e-01]
 [1.11941306e-04 5.69030729e-01 4.30857329e-01]
 [8.15661004e-01 1.69410852e-01 1.49281440e-02]]
[1. 0. 0.] [[4.39394966e-01 4.34463472e-01 1.26141562e-01]
 [4.08376491e-02 8.69254355e-01 8.99079960e-02]
 [5.14226930e-01 4.85773070e-01 1.33246438e-15]]
[1. 0. 0.] [[6.16385760e-01 3.78723287e-01 4.89095368e-03]
 [2.03904867e-04 4.53233135e-01 5.46562960e-01]
 [6.85330351e-01 2.89718187e-01 2.49514626e-02]]
[1. 0. 0.] [[0.01268052 0.02084756 0.96647192]
 [0.18452821 0.34351335 0.47195844]
 [0.01202993 0.40067129 0.58729878]]
[1. 0. 0.] [[6.68268590e-01 1.60672974e-03 3.30124680e-01]
 [3.47177950e-10 2.81898983e-48 1.0000000

[1. 0. 0.] [[2.38811568e-01 7.61188431e-01 5.49819901e-10]
 [2.16535382e-04 8.84444433e-01 1.15339031e-01]
 [9.99994978e-01 5.02165498e-06 2.90926594e-49]]
[1. 0. 0.] [[7.22637298e-01 2.77362702e-01 8.40455276e-27]
 [2.72496428e-02 9.51789318e-01 2.09610390e-02]
 [7.82891510e-11 1.00000000e+00 0.00000000e+00]]
[1. 0. 0.] [[1.25779885e-02 6.08378648e-01 3.79043363e-01]
 [1.03539304e-01 6.63916765e-01 2.32543931e-01]
 [9.97940137e-02 9.00203905e-01 2.08083600e-06]]
[1. 0. 0.] [[7.49528347e-01 1.31388493e-09 2.50471652e-01]
 [5.86461922e-11 9.99999491e-01 5.09208485e-07]
 [7.77840225e-01 1.95349552e-01 2.68102227e-02]]
[1. 0. 0.] [[1.77532832e-36 1.00000000e+00 6.93560409e-34]
 [1.81572527e-06 9.81479632e-01 1.85185521e-02]
 [1.00000000e+00 8.73021353e-15 0.00000000e+00]]
[1. 0. 0.] [[1.63186064e-10 9.74225665e-01 2.57743351e-02]
 [5.03084523e-03 6.45716828e-01 3.49252326e-01]
 [2.21272891e-01 7.59283216e-01 1.94438936e-02]]
[1. 0. 0.] [[4.54545449e-01 5.45454551e-01 3.92302990e-10]
 [3.9

[1. 0. 0.] [[1.29455393e-01 8.70101630e-01 4.42976946e-04]
 [6.92565686e-04 4.10827209e-01 5.88480225e-01]
 [6.88716740e-01 3.00828485e-01 1.04547754e-02]]
[1. 0. 0.] [[3.96675086e-01 6.03323570e-01 1.34378376e-06]
 [2.37811175e-05 7.04422198e-01 2.95554021e-01]
 [9.98064414e-01 1.87940427e-03 5.61816055e-05]]
[1. 0. 0.] [[0.39577023 0.60230071 0.00192906]
 [0.00099419 0.76067155 0.23833427]
 [0.73121461 0.25532232 0.01346306]]
[1. 0. 0.] [[3.49075944e-03 9.73775082e-01 2.27341582e-02]
 [6.33044413e-05 4.19909138e-01 5.80027558e-01]
 [8.82693453e-01 1.10642546e-01 6.66400117e-03]]
[1. 0. 0.] [[9.10452987e-01 8.29279938e-02 6.61901900e-03]
 [4.93264832e-09 9.68341107e-01 3.16588879e-02]
 [9.60248850e-01 3.07808404e-04 3.94433412e-02]]
[1. 0. 0.] [[0.91664687 0.08163416 0.00171897]
 [0.00522285 0.61424974 0.3805274 ]
 [0.13454823 0.84493728 0.02051449]]
[1. 0. 0.] [[6.75042174e-02 5.87672986e-01 3.44822797e-01]
 [1.20955917e-04 5.06938116e-01 4.92940928e-01]
 [1.75262793e-01 8.23094393e-

[1. 0. 0.] [[3.36675693e-01 6.61977481e-01 1.34682649e-03]
 [2.40352701e-04 4.22495490e-01 5.77264157e-01]
 [6.94391748e-01 2.90495115e-01 1.51131369e-02]]
[1. 0. 0.] [[1.96410050e-04 2.21385952e-01 7.78417638e-01]
 [4.71033562e-05 9.66872718e-01 3.30801784e-02]
 [9.98111277e-01 1.88738242e-03 1.34097469e-06]]
[1. 0. 0.] [[9.10612957e-02 9.03687080e-01 5.25162467e-03]
 [9.91889616e-05 1.82829761e-01 8.17071050e-01]
 [5.29215281e-01 4.70547122e-01 2.37597302e-04]]
[1. 0. 0.] [[3.34865741e-01 6.65134259e-01 1.16288750e-27]
 [8.02709011e-04 9.35282253e-01 6.39150375e-02]
 [7.49997544e-01 2.45596943e-06 2.50000000e-01]]
[1. 0. 0.] [[9.76744773e-01 1.21528073e-14 2.32552267e-02]
 [2.07920242e-04 9.99792080e-01 1.82608785e-34]
 [8.32988463e-46 1.00000000e+00 0.00000000e+00]]
[1. 0. 0.] [[7.77451559e-01 1.74388219e-01 4.81602222e-02]
 [6.61755899e-06 9.32712763e-01 6.72806199e-02]
 [8.50689381e-01 1.29173330e-01 2.01372884e-02]]
[1. 0. 0.] [[2.87151865e-01 5.27515790e-01 1.85332346e-01]
 [6.4

[1. 0. 0.] [[7.52800091e-01 1.78736832e-01 6.84630775e-02]
 [8.39125608e-05 8.25225405e-01 1.74690682e-01]
 [3.16794664e-01 1.61243692e-01 5.21961644e-01]]
[1. 0. 0.] [[8.27484828e-01 2.09283187e-02 1.51586853e-01]
 [9.08378614e-16 0.00000000e+00 1.00000000e+00]
 [9.89281762e-01 7.55311083e-43 1.07182377e-02]]
[1. 0. 0.] [[6.91056060e-01 3.08942921e-01 1.01905800e-06]
 [8.66278601e-05 7.78717401e-01 2.21195971e-01]
 [4.77615454e-01 5.22384519e-01 2.65349818e-08]]
[1. 0. 0.] [[1.86641926e-10 6.37870686e-01 3.62129314e-01]
 [4.95101414e-04 7.22249716e-01 2.77255183e-01]
 [2.42776475e-01 6.45647172e-01 1.11576353e-01]]
[1. 0. 0.] [[8.69793922e-01 1.17347073e-01 1.28590054e-02]
 [7.58848040e-05 6.40274331e-01 3.59649784e-01]
 [4.07364320e-01 5.92610229e-01 2.54514850e-05]]
[1. 0. 0.] [[9.21156263e-01 7.88437367e-02 2.50483242e-28]
 [1.01326285e-01 8.30223362e-01 6.84503528e-02]
 [9.99997836e-01 2.16393162e-06 0.00000000e+00]]
[1. 0. 0.] [[1.07479244e-06 9.99998925e-01 1.42978475e-17]
 [4.0

In [156]:

print(model.startprob_,model.transmat_)

[1. 0. 0.] [[8.68271349e-01 2.07625967e-02 1.10966055e-01]
 [3.45970946e-06 0.00000000e+00 9.99996540e-01]
 [7.93151748e-01 3.93712194e-07 2.06847858e-01]]


In [ ]:
model.predict